<a href="https://colab.research.google.com/github/SaumyaGupta-99/menudata_rag/blob/main/MenuDataRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing libraries needed
!pip install python-dotenv faiss-cpu openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [2]:
# Mounting my drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import pandas as pd
import re
import os
import numpy as np
import faiss
import torch
from transformers import AutoTokenizer, AutoModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai

In [4]:
current_working_directory = os.getcwd()
csv_file = current_working_directory + '/drive/MyDrive/Sample_Ingredients_File-MenuData_Mission_District_Sample_File-Sheet1.csv'

In [5]:
env_path = os.path.join(current_working_directory, "drive/MyDrive/faiss_index/secrets.txt")
load_dotenv(env_path)

open_ai_key = os.getenv('API_KEY')

In [6]:
web_urls= ["https://en.wikipedia.org/wiki/Ingredient#:~:text=In%20a%20general%20sense%2C%20an,its%20use%20in%20different%20recipes.",
 "https://www.masterclass.com/articles/ingredients-for-traditional-indian-cooking",
 "https://www.tastingtable.com/689854/10-essential-ingredients-for-indian-cooking/",
 "https://www.allrecipes.com/article/essential-italian-pantry-ingredients/",
 "https://en.wikipedia.org/wiki/American_cuisine",
 "https://en.wikipedia.org/wiki/Indian_cuisine",
 "https://en.wikipedia.org/wiki/Italian_cuisine",
 "https://www.tasteofhome.com/collection/desserts-from-around-the-world/?srsltid=AfmBOopJExfZcF137ou1NsvEQim6asBJinKh4KtxxGge047IsEicDTH4",
 "https://www.cnn.com/travel/article/world-50-best-desserts/index.html",
 "https://www.gazirestaurant.com.au/history-and-origins-of-heritage-cuisine-cultural-and-historical-significance-of-these-dishes",
 "https://library.fiveable.me/international-food-culture/unit-8/iconic-national-dishes-cultural-significance/study-guide/W4qLi90TWK1AeLbV",
 "https://letseattheworld.com/understanding-culture-through-food/",
 "https://www.slofoodbank.org/food-as-culture/",
 "https://thespanishgroup.org/blog/cuisine-and-culture-how-food-reflects-societies-around-the-globe/",
 "https://priceonomics.com/what-are-the-defining-ingredients-of-a-cultures/",
 "https://www.connerprairie.org/blog-what-food-reveals-about-ourselves/",
 "https://www.connerprairie.org/blog-what-food-reveals-about-ourselves/",
 "https://en.wikipedia.org/wiki/Fusion_cuisine",
 "https://en.wikipedia.org/wiki/Sephardic_Jewish_cuisine",
 "https://www.allrecipes.com/hersheys-frozen-cake-bites-walmart-8781140",
 "https://www.newyorker.com/magazine/2025/02/10/trend-alert-raw-milk-luxe-travel-and-knitting",
 "https://people.com/we-tried-starbucks-new-blackberry-sage-refreshers-8784889",
 "https://www.foodandwine.com/frozen-grapes-sour-8783516",
 "https://www.bhg.com/mccormick-flavor-of-the-year-2025-8783787",
 "https://www.allrecipes.com/mccormick-launching-aji-amarillo-seasoning-8780854",
 "https://www.foodbusinessnews.net/articles/27365-flavors-to-watch-in-2025",
 "https://www.tripadvisor.com/Restaurants-g60745-Boston_Massachusetts.html",
 "https://www.preparedfoods.com/articles/130122-six-ingredient-trends-to-keep-an-eye-on-in-2025",
 "https://www.ift.org/news-and-publications/food-technology-magazine/issues/2024/october/features/outlook-2025-flavor-trends",
 "https://sf.eater.com/maps/best-restaurants-san-francisco-38",
 "https://www.tripadvisor.com/Restaurants-g60713-San_Francisco_California.html",
 "https://www.yelp.com/search?find_desc=Impossible+Burger&find_loc=Los+Angeles%2C+CA",
 "https://la.eater.com/maps/beyond-meat-impossible-burgers-los-angeles-map",
 "https://www.findmeglutenfree.com/us/ma/boston/pizza",
 "https://www.sfchronicle.com/food/restaurants/article/bay-area-dessert-ice-19777861.php",
 "https://sfstandard.com/2024/12/09/the-best-pastries-at-sfs-hottest-new-bakeries/",
 "https://www.instagram.com/kulfipdx/reel/DE-z8tvzcEz/saffron-pistachio-our-top-selling-ice-cream-flavor-of-2024-%EF%B8%8Fwith-a-dreamy-blend-/",
 "https://www.nrn.com/food-trends/sorting-out-saffron"
 ]

In [7]:
# Scrape Website Content
def scrape_website(url):
    """Extracts text from a webpage."""
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code != 200:
        return None
    soup = BeautifulSoup(response.text, "html.parser")
    return soup.get_text()

In [8]:
# Function to process each row into a structured text format
def format_row_for_embedding(row):
    """Formats a row from the dataframe into a clean text chunk for embeddings."""
    structured_text = f"""
    Restaurant Name: {row['restaurant_name']}
    Menu Category: {row['menu_category']}
    Menu Item: {row['menu_item']}
    Description: {row['menu_description']}
    Ingredients: {row['ingredient_name']}
    Categories: {row['categories']}
    Address: {row['address1']}, {row['city']}, {row['state']}, {row['zip_code']}, {row['country']}
    Rating: {row['rating']}
    Review Count: {row['review_count']}
    Price: {row['price']}
    """
    return structured_text.strip()

In [9]:
def clean_text(text):
    """Cleans text by removing unnecessary characters, extra spaces, and normalizing."""
    if not text:
        return ""

    # Remove extra whitespaces, new lines, and unwanted characters
    text = re.sub(r'\s+', ' ', text).strip()  # Collapse multiple spaces into one
    text = re.sub(r'\n+', ' ', text)  # Remove excessive newlines
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remove non-ASCII characters (optional)

    return text

In [10]:
def fetch_data():
  df = pd.read_csv(csv_file)
  df = df.fillna("")
    # Apply function to all rows to create text chunks
  df["embedding_text"] = df.apply(format_row_for_embedding, axis=1)

  # Convert to a list of documents (each row becomes a text entry)
  documents = df["embedding_text"].tolist()

  scrape_content = []
  for url in web_urls:
    scraped_content = scrape_website(url)
    scrape_content.append(scraped_content)

  scrape_content = [clean_text(scrape_website(url)) for url in web_urls if scrape_website(url)]
  return scrape_content, documents

In [11]:
def get_embedding(text):
    """Generate embeddings using BGE."""

    model_name = "BAAI/bge-small-en"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# Process and chunk text with source information
def process_and_chunk(data, source_type):
    """
    Process text data into chunks with source information.

    Args:
        data (list): List of (text, source) tuples.
        source_type (str): Either 'url' or 'csv'.

    Returns:
        list: List of dictionaries with chunked text and source info.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Adjust as needed
        chunk_overlap=100,
        separators=["\n\n", "\n", " ", ""]
    )

    chunks_with_sources = []
    for item in data:
        text, source = item
        chunks = text_splitter.split_text(text)
        for i, chunk in enumerate(chunks):
            chunks_with_sources.append({
                "chunk": chunk,
                "source": source,
                "chunk_id": f"{source_type}_{source}_chunk_{i}"
            })
    return chunks_with_sources

def fetch_faiss_index_and_metadata():
  scrape_content, documents = fetch_data()
  documents_data = [(text, f"CSV_Row_{index}") for index, text in enumerate(documents)]
  scrape_data = [(content, url) for content, url in zip(scrape_content, web_urls)]

  csv_chunks = process_and_chunk(documents_data, source_type="csv")
  web_chunks = process_and_chunk(scrape_data, source_type="url")

  all_chunks = csv_chunks + web_chunks

  index_path = os.path.join(current_working_directory, "drive/MyDrive/faiss_index")
  index_file = os.path.join(index_path, "index_with_citation.faiss")
  metadata_file = os.path.join(index_path, "metadata.npy")

  # Create or load index
  if os.path.exists(index_file):
      index = faiss.read_index(index_file)
      print("Loaded existing FAISS index.")
      metadata = np.load(metadata_file, allow_pickle=True)
      print("Loaded existing metadata file")

  else:
      chunk_embeddings = np.array([get_embedding(chunk['chunk']) for chunk in all_chunks], dtype="float32")
      index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
      index.add(chunk_embeddings)
      faiss.write_index(index, index_file)
      print("Created and saved new FAISS index.")

      metadata = [{"source": chunk['source'], "chunk_id": chunk['chunk_id']} for chunk in all_chunks]
      np.save(metadata_file, metadata)
      print("Index and metadata saved successfully.")
  return index, metadata, all_chunks

FUNCTION TO BE CALLED WITH QUERY

In [12]:
def get_rag_response(query , index, metadata, all_chunks, open_ai_key):
  openai.api_key = open_ai_key

  query_embedding = np.array([get_embedding(query)], dtype="float32")

  D, I = index.search(query_embedding, k=10)

  retrieved_documents = []

  if isinstance(metadata, np.ndarray):
      metadata = metadata.tolist()

  for idx in I[0]:
      if idx >=len(all_chunks):
        continue
      chunk_info = all_chunks[idx]
      source_info = metadata[idx]

      retrieved_doc = {
          "text": chunk_info['chunk'],
          "source": source_info['source'],
      }

      retrieved_documents.append(retrieved_doc)

  prompt = f"Question: {query}\n\n"
  prompt += "Relevant Information:\n"
  for i, doc in enumerate(retrieved_documents):
      prompt += f"[{i+1}] {doc}\n"

  # Adding instruction to generate the response in a JSON format with citations
  prompt += """
  You read the 'Relevant Information' provided, and answer the user question. Provide your answer in a JSON format with citations.

  You return a JSON object with the following structure:
  {
      answer: string,
      quote_list: [
          {
              passage_num: int,
              quote: string,
              reason: string,
              citation: string

          }
      ]
  }.

  Explanation of the JSON:
  quote_list - an array of quotes from the context that answers the question.
  reason - a short explanation why this quote answers the question.
  answer - synthesizing your full answer, with citations based on the quotes.
  citation - the URL or reference source of the quote coming from Relevant Information (we have stored citation as source in Relevant Information).
  """

  # Use the v1/chat/completions endpoint for chat models like GPT-3.5 Turbo
  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",  # You can replace this with "gpt-4" or other models, "gpt-3.5-turbo" is typically cheaper
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt},
      ],
      max_tokens=4096,
  )

  return (response['choices'][0]['message']['content'].strip())


In [13]:
index, metadata, all_chunks = fetch_faiss_index_and_metadata()

Loaded existing FAISS index.
Loaded existing metadata file


In [ ]:
query = "Find restaurants near me that serve gluten-free pizza."
response = get_rag_response(query,index, metadata, all_chunks, open_ai_key)
print(response)

```json
{
    "answer": "There is a restaurant named Vega located at 419 Cortland Ave, San Francisco, CA, 94110.0, US, that serves gluten-free pizza.",
    "quote_list": [
        {
            "passage_num": 7,
            "quote": "Restaurant Name: vega\n    Menu Category: dinner\n    Menu Item: gluten free crust\n    Description: \n    Ingredients: gluten free flour\n    Categories: Italian|Pizza\n    Address: 419 Cortland Ave, San Francisco, CA, 94110.0, US\n    Rating: 4.1\n    Review Count: 729.0\n    Price: $$",
            "reason": "This quote provides specific information about a restaurant named Vega that serves gluten-free pizza.",
            "citation": "https://www.tripadvisor.com/Restaurants-g60745-Boston_Massachusetts.html"
        }
    ]
}
```


In [ ]:
query = "How has the use of saffron in desserts changed over the last year, according to restaurant menus or news articles?"
response = get_rag_response(query,index, metadata, all_chunks, open_ai_key)
print(response)

{
    "answer": "The use of saffron in desserts seems to have maintained its popularity and traditional associations, as there are no specific mentions of significant changes over the last year in restaurant menus or news articles.",
    "quote_list": [
        {
            "passage_num": 2,
            "quote": "Kouign Amann, Brittany, France ... Once a little-known treat that drew pastry pilgrims to Brittany, the kouign amann has officially made it big.",
            "reason": "This quote highlights the rise in popularity of a traditional dessert, suggesting that saffron desserts may also be maintaining their appeal without major changes.",
            "citation": "https://www.tasteofhome.com/collection/desserts-from-around-the-world/?srsltid=AfmBOopJExfZcF137ou1NsvEQim6asBJinKh4KtxxGge047IsEicDTH4"
        },
        {
            "passage_num": 4,
            "quote": "Cardamom Buns, Sweden ... While a freshly baked cardamom bun is a memorable treat, it\u2019s also a simple and co

In [ ]:
query = "define indian cusine?"
response = get_rag_response(query,index, metadata, all_chunks, open_ai_key)
print(response)

{
    "answer": "Indian cuisine consists of a variety of regional and traditional cuisines native to the Indian subcontinent. It is influenced by the diversity in soil, climate, culture, and occupations, resulting in substantial variations. Indian food heavily relies on locally available spices, herbs, vegetables, and fruits, with influences from Hinduism, Islam, and historical events like colonialism and trade relations. The introduction of new vegetables and fruits like potatoes, tomatoes, chillies, peanuts, and guava has become staples, shaping the history of international relations through the spice trade between India and Europe.",
    "quote_list": [
        {
            "passage_num": 1,
            "quote": "Indian cuisine consists of a variety of regional and traditional cuisines native to the Indian subcontinent.",
            "reason": "Defines what Indian cuisine consists of.",
            "citation": "https://en.wikipedia.org/wiki/American_cuisine"
        },
        {
  

In [ ]:
query = "what are the ingredients in american food?"
response = get_rag_response(query,index, metadata, all_chunks, open_ai_key)
print(response)

{
    "answer": "American cuisine consists of a wide range of styles and influences from different regions and cultures. Some popular American foods include BBQ from places like Alabama, Kansas City, and Texas, soul food, fast food, and fusion dishes. Ingredients commonly used in American food include a variety of meats like beef, pork, and chicken, seafood, pasta, fruits, vegetables, as well as different spices and flavorings.",
    "quote_list": [
        {
            "passage_num": 1,
            "quote": "American cuisine consists of the cooking style",
            "reason": "Mentions the broad scope of American cuisine incorporating various cooking styles.",
            "citation": "https://www.allrecipes.com/article/essential-italian-pantry-ingredients/"
        },
        {
            "passage_num": 8,
            "quote": "Another example of popularized fusion foods is the Korean stew budae-jjigae, which was created by combining American ingredients of Spam, Vienna sausages, 

In [ ]:
query = "list a few famous food places in Boston"
response = get_rag_response(query,index, metadata, all_chunks, open_ai_key)
print(response)

{
    "answer": "Some famous food places in Boston include Donatos, Fogo de Ch\u00e3o, and The Granola Bar.",
    "quote_list": [
        {
            "passage_num": 2,
            "quote": "Pizza chain Donatos pairs consistency, innovation as it pursues franchise growth",
            "reason": "Donatos is mentioned as a famous food place in Boston that focuses on consistency and innovation.",
            "citation": "https://www.tripadvisor.com/Restaurants-g60745-Boston_Massachusetts.html"
        },
        {
            "passage_num": 2,
            "quote": "How Brazilian steakhouse brand Fogo de Ch\u00e3o surged onto America s favorite chain list",
            "reason": "Fogo de Ch\u00e3o is highlighted as a renowned food spot in Boston that has become one of America's favorite chains.",
            "citation": "https://www.tripadvisor.com/Restaurants-g60745-Boston_Massachusetts.html"
        },
        {
            "passage_num": 2,
            "quote": "Cofounders of modern di

In [ ]:
query = "list key ingredients of a tiramisu"
response = get_rag_response(query,index, metadata, all_chunks, open_ai_key)
print(response)

```json
{
    "answer": "Key ingredients of a tiramisu typically include cocoa powder, eggs, sugar, mascarpone cheese, espresso, coffee liqueur, and ladyfingers.",
    "quote_list": [
        {
            "passage_num": 1,
            "quote": "Ingredients: cocoa powder",
            "reason": "This quote mentions cocoa powder as an ingredient in tiramisu.",
            "citation": "CSV_Row_50820"
        },
        {
            "passage_num": 2,
            "quote": "Ingredients: eggs",
            "reason": "This quote lists eggs as an ingredient used in tiramisu.",
            "citation": "CSV_Row_50822"
        },
        {
            "passage_num": 3,
            "quote": "Ingredients: sugar",
            "reason": "Sugar is identified as one of the ingredients in tiramisu in this quote.",
            "citation": "CSV_Row_50826"
        },
        {
            "passage_num": 4,
            "quote": "Ingredients: mascarpone cheese",
            "reason": "The mention of mascarp

In [15]:
query = "Define fusion cuisine with examples from local menus."
response = get_rag_response(query,index, metadata, all_chunks, open_ai_key)
print(response)

{
    "answer": "Fusion cuisine blends ingredients and cooking techniques from different countries or regions to create a new type of cuisine. It can be created by chefs or occur naturally, resulting in dishes like pancit palabok from Filipino cuisine, which combines Chinese rice noodles and tofu with Mexican annatto seeds, Spanish chicharon, and native smoked fish flakes. Fusion cuisine examples from local menus include dishes like mixed vegetables with onions and broccoli at The Spice Jar in San Francisco, categorized under Asian Fusion.",
    "quote_list": [
        {
            "passage_num": 1,
            "quote": "The term fusion cuisine is defined as 'a style of cookery which blends ingredients and methods of preparation from different countries, regions, or ethnic groups; food cooked in this style.' Fusion food is created by combining various cooking techniques from different cultures to produce a new type of cuisine.",
            "reason": "Defines fusion cuisine and explai

In [17]:
query = "Has there been an increase in the use of plant-based meat alternatives in restaurant menus this year?"
response = get_rag_response(query,index, metadata, all_chunks, open_ai_key)
print(response)

{
    "answer": "Yes, there has been an increase in the use of plant-based meat alternatives in restaurant menus this year.",
    "quote_list": [
        {
            "passage_num": 7,
            "quote": "Restaurant Name: vega Menu Category: dinner Menu Item: pizza vegetariana Description: homemade tomato sauce, mozzarella, mushrooms, zucchini, caramelized onions, olives, arugula. ... sub vegan cheese $1 Ingredients: mushrooms...",
            "reason": "This quote mentions the availability of vegan cheese as a substitution option, indicating an increase in plant-based alternatives.",
            "citation": "CSV_Row_49928"
        },
        {
            "passage_num": 10,
            "quote": "Restaurant Name: vega Menu Category: dinner Menu Item: pizza vegetariana Description: homemade tomato sauce, mozzarella, mushrooms, zucchini, caramelized onions, olives, arugula. ... sub vegan cheese $1 Ingredients: farm egg...",
            "reason": "This quote highlights the option to su

In [18]:
query = "Whats the weather in Boston?"
response = get_rag_response(query,index, metadata, all_chunks, open_ai_key)
print(response)

```json
{
    "answer": "I'm sorry, but there is no information about the current weather in Boston in the provided context.",
    "quote_list": []
}
```
